## **Summarised Plot Notebook**

In [18]:
import pandas as pd
import numpy as np 
from covid import Covid
from plotly.offline import iplot,plot,download_plotlyjs,init_notebook_mode
import cufflinks as cf
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import folium
import re
from datetime import date
from datetime import datetime, timedelta
import sys
import time
import pickle
from pathlib import Path
import base64

In [5]:

# Johns Hopkins CSSE Data (Auto-Updated)
confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
aggregate_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv'


In [6]:
def Data_load(confirmed_url,deaths_url,recovered_url,aggregate_url):
    confirmed_df = pd.read_csv(confirmed_url)
    recovered_df = pd.read_csv(recovered_url)
    deaths_df = pd.read_csv(deaths_url)
    aggregate_df = pd.read_csv(aggregate_url)

    confirmed_df = confirmed_df.replace(np.nan,'unregistered',regex=True)
    recovered_df = recovered_df.replace(np.nan,'unregistered',regex=True)
    deaths_df = deaths_df.replace(np.nan,'unregistered',regex=True)

    return confirmed_df, deaths_df, recovered_df, aggregate_df

In [7]:
confirmed_df, deaths_df, recovered_df, agg_df = Data_load(confirmed_url,deaths_url,recovered_url,aggregate_url)

In [15]:
def RAISE_in_Cases(confirmed_df,CountryName='India'):
    raise_cal = confirmed_df.copy(deep=True)
    raise_cal = raise_cal.drop(['Province/State','Lat','Long'],axis=1)
    raise_cal = raise_cal.drop(np.array(raise_cal.columns)[1:-2],axis=1)
    raise_cal = raise_cal.set_index('Country/Region')
    raise_cal['RAISE'] = np.abs(raise_cal[np.array(raise_cal.columns)[-2:][1]]- raise_cal[np.array(raise_cal.columns)[-2:][0]] )
    net_increase = raise_cal['RAISE'].sum()
    # try, catch
    if len(raise_cal.loc[CountryName,:]) > 3:
        country_raise = dict(raise_cal.loc[CountryName,:].sum())['RAISE']
    
    else:
        country_raise = dict(raise_cal.loc[CountryName,:])['RAISE']
    
    
    return net_increase,raise_cal.columns[:2],country_raise



In [16]:

def province_check_confirmed(CountryName,confirmed_df):

    if len(confirmed_df[confirmed_df['Country/Region'] == CountryName]) > 1:
        transformed_confirmed = pd.DataFrame(confirmed_df[confirmed_df['Country/Region'] ==CountryName].sum()).T
        transformed_confirmed.iat[0,1] = CountryName
        return transformed_confirmed 

    else:
        return confirmed_df

def province_check_recovered(CountryName,recovered_df):

    if len(recovered_df[recovered_df['Country/Region'] == CountryName]) > 1:
        transformed_recovered = pd.DataFrame(recovered_df[recovered_df['Country/Region'] ==CountryName].sum()).T
        transformed_recovered.iat[0,1] = CountryName
        return transformed_recovered

    else:
        return recovered_df

def province_check_deaths(CountryName,deaths_df):

    if len(deaths_df[deaths_df['Country/Region'] == CountryName]) > 1:
        transformed_deaths = pd.DataFrame(deaths_df[deaths_df['Country/Region'] ==CountryName].sum()).T
        transformed_deaths.iat[0,1] = CountryName
        return transformed_deaths

    else:
        return deaths_df

In [19]:
agg_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2021-03-25 09:26:53,33.93911,67.709953,56192.0,2466.0,49937.0,3789.0,144.347276,NaN,NaN,4.388525,4,AFG
1,Albania,2021-03-25 09:26:53,41.15330,20.168300,122295.0,2171.0,87193.0,32931.0,4249.600389,NaN,NaN,1.775216,8,ALB
2,Algeria,2021-03-25 09:26:53,28.03390,1.659600,116438.0,3069.0,81065.0,32304.0,265.530742,NaN,NaN,2.635737,12,DZA
3,Andorra,2021-03-25 09:26:53,42.50630,1.521800,11638.0,114.0,11082.0,442.0,15062.447421,NaN,NaN,0.979550,20,AND
4,Angola,2021-03-25 09:26:53,-11.20270,17.873900,21836.0,532.0,20172.0,1132.0,66.438940,NaN,NaN,2.436344,24,AGO


In [24]:
attrs = ['Confirmed','Deaths','Recovered','Active']
choropleth = []
for attr in attrs:
    choropleth.append(agg_df[['Country_Region', 'Lat','Long_','{}'.format(attr),'UID','ISO3']])

In [25]:
choro_conf, choro_death, choro_recov, choro_active = choropleth

In [56]:
choro_active

,Country_Region,Lat,Long_,Active,UID,ISO3
0,Afghanistan,33.939110,67.709953,3789.0,4,AFG
1,Albania,41.153300,20.168300,32931.0,8,ALB
2,Algeria,28.033900,1.659600,32304.0,12,DZA
3,Andorra,42.506300,1.521800,442.0,20,AND
4,Angola,-11.202700,17.873900,1132.0,24,AGO
...,...,...,...,...,...,...
187,Vietnam,14.058324,108.277199,276.0,704,VNM
188,West Bank and Gaza,31.952200,35.233200,24235.0,275,PSE
189,Yemen,15.552727,48.516388,1326.0,887,YEM
190,Zambia,-13.133897,27.849332,2189.0,894,ZMB


In [ ]:
['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
             'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
             'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
             'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
             'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
             'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
             'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
             'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
             'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
             'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
             'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
             'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
             'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
             'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
             'ylorrd'].

124838531.0

In [193]:
def choroplethChart(data,attr,config):
    total = data[attr].sum()
    
    palette = config['palette']
    border = config['border']
    title = config['title']
    
    fig = go.Figure(data=go.Choropleth(
        locations = data['ISO3'],
        z = data[attr],
        text = data['Country_Region'],
        colorscale = palette,
        autocolorscale=False,
        reversescale=True,
        marker_line_color=border,
        marker_line_width=1,
      
))

    fig.update_layout(
        title_text='{}, WorldWide {:,}'.format(attr,total),
        title_font_family='poppins',
        title_x=0.5,
        title_font_color=title,
        geo=dict(
            showframe=False,
            showcoastlines=False,
            projection_type='kavrayskiy7'
        ),
        annotations = [dict(
            x=0.55,
            y=0.1,
            xref='paper',
            yref='paper',
            text='',
            showarrow = False,
        )]
    )

    return fig
    

In [194]:
r_col = {'palette':'Tealgrn','border':'black','title':'teal'}
c_col = {'palette':'OrRd','border':'black','title':'black'}
a_col = {'palette':'PuBu','border':'teal','title':'dodgerblue'}
d_col = {'palette':'RdGy','border':'indianred','title':'crimson'}

In [195]:
fig = choroplethChart(choro_death,'Deaths',config=d_col)
fig.show()

***